In [4]:
a = [66, 666, 6666]
a = None

b = [1, 2, *(a or []), 3, 4]

b

[1, 2, 3, 4]

In [6]:
from typing import List

from langchain.chains.ernie_functions import create_structured_output_chain
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_core.messages import AIMessage, SystemMessage

system_message = (
    'You are a helpful GIS agent/consultant.\n'
    'Table names should be surrounded in double quotes.\n'
    'DO NOT waste time presenting schemas and example rows to the user.\n'
)

class AISuffix(BaseModel):
    ai_suffix: str = Field(
        ..., description="The next thought of the AI agent, meant to help guide it solve the problem at hand")


def select_ai_suffix_message(chat_history: List[BaseMessage], user_query: str):
    available_ai_suffixes = [
        (
            "I should look at the tables in the database to see what I can query.\n"
            "Then I should query the schema of the most relevant tables, before doing an SQL query to answer the user's request.\n"
            "If no relevant data is found in the database, I should use my background knowledge to give an approximate answer.\n"
        ),
        ['']
    ]

    llm = ChatOpenAI()
    prompt = ChatPromptTemplate.from_template(
        [
            (
                'You are a supervisor for a language model.\n'
                'Based on a list of messages between a human user and an AI agent, '
                'you will help decide a strategy for the AI agent by selecting its next "thought".\n'
                'You will select between different strategies that are provided to you. '
                'If none of these fit the conversation, set the AI thought to an empty string ("").\n\n'
                'Here is the system message for the AI agent:\n\n"""{system_message}"""'
                'Here is the conversation up to this point:\n\n{chat_history}\n\n'
                'Here here are the available ai_suffixes for you to choose from:\n\n{available_ai_suffixes}'
            )
        ]
    )
    prompt.format(chat_history=chat_history,
                  available_ai_suffixes=available_ai_suffixes)

    chain = create_structured_output_chain(AISuffix, llm, prompt)
    return chain.invoke({})

In [78]:
from langchain_core.messages import AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, streaming=True)

system_message = (
    'You are a helpful GIS agent/consultant.\n'
    'Table names should be surrounded in double quotes.\n'
    'DO NOT waste time presenting schemas and example rows to the user.\n'
)


prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=(
            'You are a helpful GIS agent/consultant.\n'
            'Table names should be surrounded in double quotes.\n'
            'DO NOT waste time presenting schemas and example rows to the user.\n'
        )),
        MessagesPlaceholder(variable_name='chat_memory'),
        HumanMessagePromptTemplate.from_template("{input}"),
        AIMessagePromptTemplate.from_template('{ai_suffix}', input_variables=['ai_suffix']),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

tools = []

agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [112]:
if not 1:
    print('hei')

In [86]:
messages = agent.get_prompts()[0].messages
[m.dict() for m in messages]

[{'content': 'You are a helpful GIS agent/consultant.\nTable names should be surrounded in double quotes.\nDO NOT waste time presenting schemas and example rows to the user.\n',
  'additional_kwargs': {},
  'type': 'system'},
 {'variable_name': 'chat_memory', 'optional': False},
 {'prompt': {'name': None,
   'input_variables': ['input'],
   'input_types': {},
   'output_parser': None,
   'partial_variables': {},
   'template': '{input}',
   'template_format': 'f-string',
   'validate_template': False,
   '_type': 'prompt'},
  'additional_kwargs': {}},
 {'prompt': {'name': None,
   'input_variables': ['ai_suffix'],
   'input_types': {},
   'output_parser': None,
   'partial_variables': {},
   'template': '{ai_suffix}',
   'template_format': 'f-string',
   'validate_template': False,
   '_type': 'prompt'},
  'additional_kwargs': {}},
 {'variable_name': 'agent_scratchpad', 'optional': False}]

In [110]:
# agent.get_prompts()
messages = agent_executor.agent.dict()['runnable']['middle'][0]['messages']
if messages[0]['type'] == 'system': 
    print(messages[0]['content'])
# agent_executor.

You are a helpful GIS agent/consultant.
Table names should be surrounded in double quotes.
DO NOT waste time presenting schemas and example rows to the user.



In [71]:
available_ai_suffixes = [
    (
        "I should look at the tables in the database to see what I can query.\n"
        "Then I should query the schema of the most relevant tables, before doing an SQL query to answer the user's request."
    ),
    "I should use my background knowledge to give an approximate answer.",
    # "I should ask for clarification from the user."
]

system_message = (
    'You are a helpful GIS agent/consultant that has access to a couple of geospatial datasets in a PostGIS database.\n'
    'Table names should be surrounded in double quotes.\n'
    'DO NOT waste time presenting schemas and example rows to the user.\n'
)

llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            (
                'You are a supervisor for a language model.\n'
                'Based on a list of messages between a human user and an AI agent, '
                'you will help decide a strategy for the AI agent by selecting its next "thought".\n'
                'You will select between different strategies that are provided to you. '
                'If none of these fit the conversation, set the AI thought to an empty string ("").\n\n'
                'Here is the system message for the AI agent:\n\n"""{system_message}"""\n\n'
                'Here is the conversation up to this point:\n\n{chat_history}\n\n'
                'Here here are the available ai_suffixes for you to choose from:\n\n{available_ai_suffixes}\n\n'
                'You are allowed to make minor adjustments to the ai_suffix you select.'
            )
        )
    ]
)
# formatted_prompt = prompt.format(...)


class AISuffix(BaseModel):
    ai_suffix: str = Field(
        ..., description="The next thought of the AI agent, meant to help guide it solve the problem at hand")


chain = create_structured_output_chain(AISuffix, llm, prompt)
res = chain.invoke({
    'system_message': system_message,
    'chat_history': [
        {
            "content": "hi",
            "additional_kwargs": {

            },
            "type": "human",
            "example": False
        },
        {
            "content": "Hello! How can I assist you today?",
            "additional_kwargs": {

            },
            "type": "ai",
            "example": False
        },
        {
            "content": "what tables do you have?",
            "additional_kwargs": {

            },
            "type": "human",
            "example": False
        },
        {
            "content": "The tables in the database are:\n1. \"5001_25832_ar50_gml\"\n2. \"5001_strategisk_stoykartlegging_dogn_25832f\"\n3. \"5001elveg2.0l\"\n4. \"Basisdata_5001_Trondheim_25832_MatrikkelenAdresse_GML\"\n5. \"Basisdata_5001_Trondheim_25832_MatrikkelenBygning_GML\"\n6. \"KartlagteFriluftslivsomr_5001_trondheim_25832_GML\"\n7. \"Kulturminner_5001_Trondheim_25832_Fredabygninger_GML\"\n8. \"Samfunnssikkerhet_5001_Trondheim_25832_Flomsoner_GML\"\n9. \"Samfunnssikkerhet_5001_Trondheim_25832_Kvikkleire_GML\"\n10. \"layer\"\n11. \"spatial_ref_sys\"\n12. \"topology\"\n\nHow can I assist you further with this information?",
            "additional_kwargs": {

            },
            "type": "ai",
            "example": False
        },
        {
            "content": "Hva er det gamle navnet på Oslo?",
            "additional_kwargs": {},
            "type": "human",
            "example": False
        }
    ],
    'available_ai_suffixes': available_ai_suffixes
})

res

{'system_message': 'You are a helpful GIS agent/consultant that has access to a couple of geospatial datasets in a PostGIS database.\nTable names should be surrounded in double quotes.\nDO NOT waste time presenting schemas and example rows to the user.\n',
 'chat_history': [{'content': 'hi',
   'additional_kwargs': {},
   'type': 'human',
   'example': False},
  {'content': 'Hello! How can I assist you today?',
   'additional_kwargs': {},
   'type': 'ai',
   'example': False},
  {'content': 'what tables do you have?',
   'additional_kwargs': {},
   'type': 'human',
   'example': False},
  {'content': 'The tables in the database are:\n1. "5001_25832_ar50_gml"\n2. "5001_strategisk_stoykartlegging_dogn_25832f"\n3. "5001elveg2.0l"\n4. "Basisdata_5001_Trondheim_25832_MatrikkelenAdresse_GML"\n5. "Basisdata_5001_Trondheim_25832_MatrikkelenBygning_GML"\n6. "KartlagteFriluftslivsomr_5001_trondheim_25832_GML"\n7. "Kulturminner_5001_Trondheim_25832_Fredabygninger_GML"\n8. "Samfunnssikkerhet_5001_

In [73]:
res['function'].ai_suffix

'I should use my background knowledge to give an approximate answer.'

In [12]:
msgs = [
    SystemMessage(content=(
        'You are a helpful GIS agent/consultant.\n'
        'Table names should be surrounded in double quotes.\n'
        'DO NOT waste time presenting schemas and example rows to the user.\n'
    )),
    MessagesPlaceholder(variable_name='chat_history'),
    HumanMessagePromptTemplate.from_template("{input}"),
]

[m.dict() for m in msgs]

[{'content': 'You are a helpful GIS agent/consultant.\nTable names should be surrounded in double quotes.\nDO NOT waste time presenting schemas and example rows to the user.\n',
  'additional_kwargs': {},
  'type': 'system'},
 {'variable_name': 'chat_history', 'optional': False},
 {'prompt': {'name': None,
   'input_variables': ['input'],
   'input_types': {},
   'output_parser': None,
   'partial_variables': {},
   'template': '{input}',
   'template_format': 'f-string',
   'validate_template': False,
   '_type': 'prompt'},
  'additional_kwargs': {}}]